In [94]:
import pandas as pd
import numpy as np
import warnings as ws
ws.filterwarnings("ignore", category=RuntimeWarning) 

In [95]:
# Importing the data and skipping the bad lines
Goalies_beforecleaning = pd.read_csv('C:/Users/ROHINI/Desktop/Personal/Futur/MOVELLA/Goalies.csv',on_bad_lines = 'skip')
Goalies_beforecleaning.to_csv('C:/Users/ROHINI/Desktop/Personal/Futur/MOVELLA/Goalies_tr.csv')
Goal_tender = pd.read_csv('C:/Users/ROHINI/Desktop/Personal/Futur/MOVELLA/Goalies_tr.csv')

In [96]:
#Dropping unrelated fields
Goal_tender.drop([ "Unnamed: 0","stint","lgID","PostGP","PostMin","PostW","PostL","PostT","PostENG","PostSHO","PostGA","PostSA"],axis=1,inplace=True)

In [97]:
#Sorting the dataframe
Goal_tender = Goal_tender.sort_values("tmID", axis = 0, ascending = True)

# Compute Wins_agg,Losses_agg,GP_agg,Mins_over_GA_agg,GA_over_SA_agg, and avg_percentage_wins

In [98]:

total_Win=Goal_tender['W'].sum(axis=0)
Goal_tender['Percentageofgameswon']=Goal_tender['W']/total_Win
Goal_tender['tofindmean']=Goal_tender['Percentageofgameswon']
Goal_tender

cols=['W','L','GP','Min','GA','SA']
cols_sum=( Goal_tender.groupby(['tmID','year'],as_index=False)[cols]
          .agg(sum)
           .reindex(columns=Goal_tender.columns) )

cols=['tofindmean']
cols_mean=( Goal_tender.groupby(['tmID','year'],as_index=False)[cols]
          .agg('mean')
           .reindex(columns=Goal_tender.columns) )


cols=['playerID']
cols_count=( Goal_tender.groupby(['tmID','year'],as_index=False)[cols]
          .agg('count')
           .reindex(columns=Goal_tender.columns) )

Goalies=pd.DataFrame([cols_sum.tmID,cols_sum.year,cols_count.playerID,
                 cols_sum.W, cols_sum.L,cols_sum.GP,cols_sum.Min,
                 cols_sum.GA,cols_sum.SA,cols_mean.tofindmean]).copy()

Goalies=Goalies.transpose()
Goalies_agg = Goalies.rename(columns={Goalies.columns[2]: 'totalPlayers',Goalies.columns[3]: 'totalWins',Goalies.columns[4]: 'totalLosses',
                         Goalies.columns[5]: 'totalGamesplayed',Goalies.columns[6]: 'totalMinutesPlayed',
                         Goalies.columns[7]: 'totalGoalsAgainst',Goalies.columns[8]: 'totalshotsAgainst',Goalies.columns[9]: 'avg_percentage_wins'
                        })
#Goalies_agg


In [99]:
#Converting datatypes for required calculations
Goalies_agg=Goalies_agg.convert_dtypes()
#Goalies_agg.dtypes

In [100]:


aggregates = {}
for tmID in Goalies_agg['tmID']:
       
        totalWins    = Goalies_agg['totalWins']
        totalPlayers = Goalies_agg['totalPlayers']
        totalLosses  = Goalies_agg['totalLosses']
        totalGamesplayed=Goalies_agg['totalGamesplayed']
        totalMinutesPlayed=Goalies_agg['totalMinutesPlayed']
        totalGoalsAgainst=Goalies_agg['totalGoalsAgainst']
        totalshotsAgainst=Goalies_agg['totalshotsAgainst']
        
        Wins_agg=totalWins/totalPlayers
        Losses_agg=totalLosses/totalPlayers
        GP_agg=totalGamesplayed/totalPlayers
        
        try:
            Mins_over_GA_agg=totalMinutesPlayed/totalGoalsAgainst
        except ZeroDivisionError:
            Mins_over_GA_agg=0
            
            
        try:
            GA_over_SA_agg=totalGoalsAgainst/totalshotsAgainst
        except ZeroDivisionError:
            GA_over_SA_agg=0
            
        
        Goalies_agg=Goalies_agg.assign(Wins_agg=Wins_agg)
        Goalies_agg=Goalies_agg.assign(Losses_agg=Losses_agg)
        Goalies_agg=Goalies_agg.assign(GP_agg=GP_agg)
        Goalies_agg=Goalies_agg.assign(Mins_over_GA_agg=Mins_over_GA_agg)
        Goalies_agg=Goalies_agg.assign(GA_over_SA_agg=GA_over_SA_agg)


# Output

In [101]:
#Keep required fields
Goalies_aggregates=Goalies_agg.copy()
Goalies_aggregates.drop([ "totalPlayers","totalWins","totalLosses","totalGamesplayed","totalMinutesPlayed","totalGoalsAgainst","totalshotsAgainst"],axis=1,inplace=True)
Goalies_aggregates


,tmID,year,avg_percentage_wins,Wins_agg,Losses_agg,GP_agg,Mins_over_GA_agg,GA_over_SA_agg
0,ALB,1972,0.000416,19.0,18.5,42.0,18.805556,0.101327
1,ANA,1993,0.000241,11.0,15.333333,30.0,20.970954,0.092479
2,ANA,1994,0.000175,8.0,13.5,28.5,18.367089,0.1
3,ANA,1995,0.000383,17.5,19.5,44.5,20.508264,0.091875
4,ANA,1996,0.000263,12.0,11.0,30.666667,21.907489,0.084638
...,...,...,...,...,...,...,...,...
1485,WIN,1991,0.000181,8.25,8.0,23.0,20.304167,0.096192
1486,WIN,1992,0.000219,10.0,9.25,22.75,16.03481,0.115118
1487,WIN,1993,0.000131,6.0,12.75,25.0,15.029586,0.12106
1488,WIN,1994,0.000175,8.0,12.5,28.0,16.820809,0.112265


#  most_goals_stopped: {‘playerID’: playerID, ‘goals_stopped’: goals_stopped} 


In [102]:
goals_stopped = {}
i=0
for playerID in Goal_tender['playerID'].unique():
        tempdf = Goal_tender[Goal_tender['playerID'] == playerID]
        goals_stopped[i]={}
        totalshutout = tempdf['SHO'].sum()
        goals_stopped[i]['playerID'] = playerID
        goals_stopped[i]['SHO']=totalshutout
        i+=1
sorted_keys = sorted(goals_stopped, key=lambda x: (goals_stopped[x]['SHO']),reverse=True)
most_goals_stoppedset={}
j=0
for i in sorted_keys:
    most_goals_stoppedset[j]={}
    k=i
    most_goals_stoppedset[j]['playerID']=goals_stopped[k]['playerID']
    most_goals_stoppedset[j]['SHO']=goals_stopped[k]['SHO']
    j+=1
    
most_goals_stopped={}

most_goals_stopped['playerID']=most_goals_stoppedset[0]['playerID']
most_goals_stopped['goals_stopped']=most_goals_stoppedset[0]['SHO']
print(most_goals_stopped)

{'playerID': 'brodema01', 'goals_stopped': 116.0}


# most_efficient_player: {‘playerID’: playerID, ‘efficiency’: goals_stopped / minutes played} 


In [103]:
Efficiency = {}
i=0
for playerID in Goal_tender['playerID'].unique():
        tempdf = Goal_tender[Goal_tender['playerID'] == playerID]
        Efficiency[i]={}
        totalshutout = tempdf['SHO'].sum()
        totalMin = tempdf['Min'].sum()
        try:        
            eff=round(totalshutout/totalMin,8)
        except ZeroDivisionError:
            eff=0     
        
        Efficiency[i]['playerID'] = playerID
        Efficiency[i]['eff']=eff
        i+=1

sorted_keys = sorted(Efficiency, key=lambda x: (Efficiency[x]['eff']),reverse=True)
most_efficient_playerset={}
j=0
for i in sorted_keys:
    most_efficient_playerset[j]={}
    k=i
    most_efficient_playerset[j]['playerID']=Efficiency[k]['playerID']
    most_efficient_playerset[j]['eff']=Efficiency[k]['eff']
    j+=1
    
most_efficient_player['playerID']=most_efficient_playerset[0]['playerID']
most_efficient_player['efficiency']=most_efficient_playerset[0]['eff']
print(most_efficient_player)       

{'playerID': 'pronocl01', 'efficiency': 0.00833333}
